In [ ]:
"""
微调模型3的推理（cosmosqa数据集）
csv是为了保存模型给出的结果和提取的答案（数字）
lst是最终提交到网站上算准确率的文件
"""
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import csv
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


for model in ['20240728042026']:
    path = f"/hy-tmp/MiniCPM/finetune/output/LoRA/{model}/checkpoint-1000"
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForCausalLM.from_pretrained(
        path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
    )
    
    input_file_path = '/hy-tmp/cosmosqa-master/data/test.jsonl'
    csv_file_path = os.path.join(path, 'predictions_cosmosqa.csv')
    lst_file_path = os.path.join(path, 'predictions_cosmosqa.lst')
    
    # 逐行读取 JSONL 文件
    with open(input_file_path, 'r') as file, open(csv_file_path, 'w', newline='') as csvfile, open(lst_file_path, 'w') as lstfile:
        # 初始化列表来存储 id 和 label
        data = []
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'label'])    
        for line in file:
            record = json.loads(line)
            # 提取 id 和 label 信息
            id = record['id']
            input = f"<用户>You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the number of the option needs to be answered and your answer is only one number.\n" \
             f"Here are the questions you have to answer: \n" \
             f"Text: {record['context']}\n" \
             f"Question: {record['question']}\n" \
             f"Option 0: {record['answer0']}\n" \
             f"Option 1: {record['answer1']}\n" \
             f"Option 2: {record['answer2']}\n" \
             f"Option 3: {record['answer3']}\n"
            res = model.chat(tokenizer, query=input, max_length=1024, top_p=0.5)[0]
            label = re.sub(r'\D', '', res)  # 提取回答中的数字
            writer.writerow([id, label])
            lstfile.write(f"{label}\n")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to